In [1]:
import sys
import tensorflow as tf
import json
import os
n_epoch = 3
import pandas as pd

In [2]:
import csv
from pandas.io.json import json_normalize
dirs = os.listdir("Downloads/json/")
print len(dirs)
with open('tripadv.csv','wb') as f:
    writer = csv.writer(f)
    writer.writerow(["Ratings","Content"])
    for filename in dirs:
        with open('Downloads/json/'+filename) as fi:
            
            try:
                d = json.load(fi)
            except:
                d={}
                d["Reviews"] = []
        for x in d["Reviews"]:
            if float(x["Ratings"]["Overall"]) in [1.0,2.0,3.0,4.0,5.0]:
                writer.writerow([float(x["Ratings"]["Overall"].encode('utf-8')),x["Content"].encode('utf-8')])

1326


In [5]:
import pandas as pd
df = pd.read_csv('out.csv')
df.head(10)

,Ratings,Content
0,5.0,"Usually stay near the airport, but this trip w..."
1,4.0,Stayed at this Hilton for 2 nights. It was lik...
2,4.0,"Stayed there one night, December 16, on the wa..."
3,5.0,I just stayed here last weekend and have alrea...
4,5.0,My mother who is 90 and I stayed one night on ...
5,4.0,"A fabulous stop. Clean, friendly, excellent lo..."
6,4.0,I stopped here just for the night on a road tr...
7,4.0,Great spot close to shops and dinning but perh...
8,4.0,My husband and I go to Baltimore often as he h...
9,5.0,Super clean. Great hotel to stay at. We actual...


In [6]:
X = df.Content
y = df.Ratings
X.head(10)
y.head(10)

0    5.0
1    4.0
2    4.0
3    5.0
4    5.0
5    4.0
6    4.0
7    4.0
8    4.0
9    5.0
Name: Ratings, dtype: float64

In [24]:
def get_positive_or_negative_classes_two_classes(y):
    return y.apply(lambda val: 0 if val<=2 else 1)
def get_positive_or_negative_classes_three_classes(y):
    return y.apply(lambda val: 0 if val<=2 else 2 if val > 3 else 1)
def get_positive_or_negative_classes_five_classes(y):
    return y.apply(lambda val: 0 if val=='' else val)

In [7]:
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
y_train = get_positive_or_negative_classes_five_classes(y_train)
y_test = get_positive_or_negative_classes_five_classes(y_test)

In [10]:
import string
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(ngram_range=(1,1), token_pattern=r'\b\w{1,}\b')

In [11]:
vect.fit(X_train)
vocab = vect.vocabulary_
len(vocab)

132362

In [8]:
def convert_X_to_X_word_ids(X):
    return X.apply( lambda x: [vocab[w] for w in [w.lower().strip() for w in x.split()] if w in vocab] )

In [15]:
X_train_word_ids = convert_X_to_X_word_ids(X_train)
X_test_word_ids  = convert_X_to_X_word_ids(X_test)


In [16]:
X_train_word_ids.head()

87742     [61869, 117272, 96328, 91452, 103347, 53053, 6...
178022    [20262, 102137, 130008, 14055, 80615, 73925, 6...
214335    [57226, 4595, 118614, 126568, 117939, 127793, ...
115399    [128249, 112180, 117939, 61869, 43304, 118986,...
45982     [128249, 55082, 86553, 85218, 37384, 10077, 12...
Name: Content, dtype: object

In [17]:
X_train.head()

87742     Ancient, Budget-Minded Hotel The Ramada Plaza ...
178022    Beautiful rooms with american movie memorabili...
214335    Great 1st Time Visit This was my first visit t...
115399    We stayed this hotel due to a flight cancellat...
45982     We gave our olders daughter a trip to New York...
Name: Content, dtype: object

In [18]:
X_train_padded_seqs = pad_sequences(X_train_word_ids, maxlen=20, value=0)
X_test_padded_seqs  = pad_sequences(X_test_word_ids , maxlen=20, value=0)

In [69]:
print('X_train_padded_seqs.shape:', X_train_padded_seqs.shape)
print('X_test_padded_seqs.shape:', X_test_padded_seqs.shape)

('X_train_padded_seqs.shape:', (249481, 20))
('X_test_padded_seqs.shape:', (27721, 20))


In [11]:
pd.DataFrame(X_train_padded_seqs).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,117940,27635,14806,100844,16499,129388,126708,63449,61002,98215,44384,17548,117940,43140,83959,117940,52758,14806,17679,129388
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,51666,63449,14170,84176,10430,28576,62322,58600,17548,25872,63749,81749,50547,23913,62322,67114,110215,87187,64574,63449
3,63749,132436,65988,85956,56872,119676,126271,63449,131441,104520,119299,59703,89264,117940,21252,39522,85956,129653,132436,118618
4,63749,132436,82930,75287,132522,128890,119676,117940,129480,132436,79726,87187,85956,10430,27635,117940,72978,130345,82930,20370


In [12]:
pd.DataFrame(X_test_padded_seqs).head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,63449,128501,74099,53247,119676,117940,57402,127950,12661,63449,58745,105785,67296,85956,117940,121126,63449,128501,84176,10684
1,81749,56404,117940,80712,85956,117897,14806,63449,23469,10430,112472,102671,17548,117940,16610,52890,87117,126271,17062,63449
2,118618,67114,10430,83179,95059,14806,10430,57663,73724,52890,126993,85396,117940,20249,14806,82292,117940,102755,16477,119519
3,90093,130345,14149,59851,63449,51666,112831,18753,84719,64574,106175,52890,8235,63449,58745,10430,131060,119299,25872,40435
4,121430,97478,96355,29180,48253,101862,64574,117940,60159,85396,78839,128107,67114,11661,117940,128176,41924,119676,102349,28296


In [19]:
unique_y_labels = list(y_train.value_counts().index)
unique_y_labels

[5.0, 4.0, 3.0, 2.0, 1.0]

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(unique_y_labels)

LabelEncoder()

In [21]:
y_train = to_categorical(y_train.map(lambda x: le.transform([x])[0]), nb_classes=len(unique_y_labels))
y_test  = to_categorical(y_test.map(lambda x:  le.transform([x])[0]), nb_classes=len(unique_y_labels))

In [41]:
y_train

array([[ 0.,  1.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.,  1.],
       [ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  1.]])

In [89]:
print('y_train.shape:', y_train.shape)
print('y_test.shape:', y_test.shape)

('y_train.shape:', (249481, 6))
('y_test.shape:', (27721, 6))


In [15]:
size_of_each_vector = X_train_padded_seqs.shape[1]
vocab_size = len(vocab)
no_of_unique_y_labels = len(unique_y_labels)

In [91]:
print('size_of_each_vector:', size_of_each_vector)
print('vocab_size:', vocab_size)
print('no_of_unique_y_labels:', no_of_unique_y_labels)

('size_of_each_vector:', 20)
('vocab_size:', 133076)
('no_of_unique_y_labels:', 6)


In [16]:
net = tflearn.input_data([None, size_of_each_vector]) # The first element is the "batch size" which we set to "None"
net = tflearn.embedding(net, input_dim=vocab_size, output_dim=128) # input_dim: vocabulary size
net = tflearn.lstm(net, 128, dropout=0.6) # Set the dropout to 0.6
net = tflearn.fully_connected(net, no_of_unique_y_labels, activation='softmax') # relu or softmax
net = tflearn.regression(net, 
                         optimizer='adam',  # adam or ada or adagrad # sgd
                         learning_rate=1e-4,
                         loss='categorical_crossentropy')

In [17]:
model = tflearn.DNN(net, tensorboard_verbose=0)

In [18]:
model.fit(X_train_padded_seqs, y_train, 
          validation_set=(X_test_padded_seqs, y_test), 
          n_epoch=n_epoch,
          show_metric=True, 
          batch_size=100)

Training Step: 7472  | total loss: 1.18473 | time: 717.630s
| Adam | epoch: 003 | loss: 1.18473 - acc: 0.4782 -- iter: 249000/249050
Training Step: 7473  | total loss: 1.18347 | time: 727.044s
| Adam | epoch: 003 | loss: 1.18347 - acc: 0.4773 | val_loss: 1.19817 - val_acc: 0.4743 -- iter: 249050/249050
--


In [21]:
import numpy as np
from sklearn import metrics

In [37]:
pred_classes = [np.argmax(i) for i in model.predict(X_test_padded_seqs)]
true_classes = [np.argmax(i) for i in y_test]
print type(model.predict(X_test_padded_seqs))
print type(pred_classes)
print type(true_classes)
print('\nRNN Classifier\'s Accuracy: %0.5f\n' % metrics.accuracy_score(true_classes, pred_classes))



<type 'numpy.ndarray'>
<type 'list'>
<type 'list'>

RNN Classifier's Accuracy: 1.00000



In [30]:
# print le.inverse_transform(pred_classes)
print ('Confusion Matrix:', metrics.confusion_matrix(le.inverse_transform(true_classes), le.inverse_transform(pred_classes)))
print ('Classification Report')
print metrics.classification_report(le.inverse_transform(true_classes), le.inverse_transform(pred_classes))

('Confusion Matrix:', array([[ 957,   25,  242,  429,  545],
       [ 447,   53,  451,  775,  617],
       [ 272,   23,  753, 1931, 1064],
       [ 169,   10,  506, 4069, 4262],
       [ 165,    2,  190, 2424, 7292]]))
Classification Report
             precision    recall  f1-score   support

        1.0       0.48      0.44      0.45      2198
        2.0       0.47      0.02      0.04      2343
        3.0       0.35      0.19      0.24      4043
        4.0       0.42      0.45      0.44      9016
        5.0       0.53      0.72      0.61     10073

avg / total       0.46      0.47      0.44     27673



In [20]:
ids_of_titles = range(0,100) # range(X_test.shape[0]) 

for i in ids_of_titles:
    pred_class = np.argmax(model.predict([X_test_padded_seqs[i]]))
    true_class = np.argmax(y_test[i])
    
    print(X_test.values[i])
    print('pred_class:', le.inverse_transform(pred_class))
    print('true_class:', le.inverse_transform(true_class))
    print('')

I was looking forward to the Grand Wailea after I had seen it on the travel channel. I was not able to stay there the entire time I was on Maui, because of price. I used Hilton pts for 2 nights. The first 2 nights on Maui were spent at the Marriott next to Grand Wailea....beautiful by the way...anyway. As we checked in, confirmation in hand, I was told there was a additional charge for having 4 people in the room. No where on my conf did it say that and I booked it for 4 people. Conf stated 4 people and 2 double beds. The woman checking us in had nothing to say except that is the rules. I asked to speak to the manager, but she just happened to step into a meeting at that exact moment. So, I asked for a phone call. Did I receive a call the 2 days we were there.....no!!!! The additional $200 they got from us, was the only money they got from us. We refused to eat or buy anything on that property. We walked to the shopping center and had the best breakfast on the island at Longhi's.......